###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2018  by D. Koehn, notebook style sheet by L.A. Barba, N.C. Clementi

###### Translated by Huizhe Di @ SCSIO

# 时间域的粘弹性运动方程

在上一课中，我们证明了广义麦克斯韦模型(GMB)能够描述恒定的Q值谱，这是黏弹性介质中真实SH波传播所必需的。现在，我们只需要将频率域的GMB应力-应变关系转换回时间域，并将正问题组装成可以用有限差分求解的形式。


## GMB应力-应变关系: 从频率域到时间域

通过将一个麦克斯韦标准线性固体(SLS)与另外的麦克斯韦体并行组合，我们导出了 **广义麦克斯韦体(GMB)**

<img src="images/GMB.png" width="50%">

它可以在频率域用应力-应变关系来描述:

\begin{equation}
\tilde{\sigma} = \tilde{\mu}_{GMB} \tilde{\epsilon} \notag
\end{equation}

和复剪切模量:

\begin{equation}
\tilde{\mu}_{GMB} = \mu_0 + \delta \mu \sum_{l=1}^{L} \frac{i a_l \omega}{i \omega + \omega_l}. \notag
\end{equation}

这里 $\mu_0$ 表示松弛剪切模量，$\delta \mu$ 为模量的缺陷, L 为Maxwell体的个数，$a_l, \omega_l$ 为权重系数，和L-麦克斯韦体的弛豫频率，以达到恒定的Q值谱，而 $\omega$ 是震源子波频率范围内的圆频率。

对于时间域的有限差分建模, 显然，我们需要通过傅里叶反变换将复模量转化为时间域，从而得到松弛函数 $\Psi(t)$

\begin{equation}
\Psi(t) = \frac{1}{2\pi}\int_{-\infty}^{\infty}\tilde{\mu}_{GMB} \exp(i\omega t) d\omega. \notag
\end{equation}

这导出

\begin{equation}
\Psi(t) = \biggl\{\mu_0 + \delta \mu \sum_{l=1}^L a_l \exp(-\omega_l t) \biggr\} \cdot H(t), \notag
\end{equation}

这里 $H(t)$ 为Heaviside step function. 用模量缺陷的定义作为非松弛模量 $\mu_u$ 和松弛剪切模量 $\mu_0$ 之间的差值:

\begin{equation}
\delta\mu = \mu_u - \mu_0\notag
\end{equation}

我们可以用非松弛模量代替松弛模量 ([Emmerich & Korn 1987](https://library.seg.org/doi/10.1190/1.1442386), [Moczo & Kristek 2004](http://www.spice-rtn.org/library/publications/publicationbook.2007-05-08.3817028446.html)):

\begin{equation}
\Psi(t) = \biggl\{\mu_u - \delta \mu \sum_{l=1}^L a_l\biggl(1 - \exp(-\omega_l t)\biggr) \biggr\} \cdot H(t). \notag
\end{equation}

在将松弛函数插入粘弹性应力-应变关系之前:

\begin{equation}
\sigma = \int_{-\infty}^{t}\dot{\Psi}(t-t')\epsilon(t')dt' = \dot{\Psi} * \epsilon\notag
\end{equation}

我们要求它的一阶导数:

\begin{equation}
\dot{\Psi}(t) = - \delta \mu \sum_{l=1}^L a_l \omega_l \exp(-\omega_l t) \cdot H(t) + \biggl\{\mu_u - \delta \mu \sum_{l=1}^L a_l\biggl(1 - \exp(-\omega_l t)\biggr) \biggr\} \cdot \delta(t), \notag
\end{equation}

和Delta函数 $\delta(t)$, 这里我们使用

\begin{equation}
\frac{\partial H(t)}{\partial t} = \delta(t)\notag
\end{equation}

将松弛函数 $\dot{\Psi}$ 的时间导数插入粘弹性应力-应变关系得到:

\begin{align}
\sigma(t)=& -\int_{-\infty}^{t}\delta\mu \sum_{l=1}^L a_l \omega_l \exp(-\omega_l (t-t'))\cdot H(t-t')\; \epsilon(t') dt'\notag\\
&+ \int_{-\infty}^{t} \mu_0 \cdot \delta (t-t') \cdot \epsilon(t') dt'\notag\\
&-\int_{-\infty}^{t}\delta\mu \sum_{l=1}^L a_l \biggl(1 - \exp(-\omega_l (t-t'))\biggr)\cdot \delta(t-t')\; \epsilon(t') dt'\notag\\
\end{align}

计算Heaviside函数和Delta函数得到:

\begin{equation}
\sigma(t) = \mu_u \cdot \epsilon(t) - \delta\mu \sum_{l=1}^L a_l \omega_l \int_{-\infty}^t\epsilon(t')\cdot \exp(-\omega_l (t-t')) dt'\notag
\end{equation}

这个表达式的问题是剩余的时间卷积，这将需要应用于FD时间步进中的每个空间网格点和每个时间步，导致巨大的计算成本。为了摆脱这个卷积，我们使用一个标准的物理学技巧:第一步，我们给这个问题起一个新名字，在这种情况下我们叫:

\begin{equation}
\zeta_l(t) = \omega_l \int_{-\infty}^t \epsilon(t') \cdot \exp(-\omega_l (t-t')) dt'\notag
\end{equation}

**内存变量** 对于 L Maxwell体 $l=1,...,L$, 引出应力-应变关系

\begin{equation}
\sigma(t) = \mu_u \cdot \epsilon(t) - \sum_{l=1}^L \delta\mu a_l \zeta_l(t),\notag
\end{equation}

这里，我们不再需要应用时间卷积了。然而，我们需要额外的偏微分方程来估计内存变量 $\zeta_l$。所以在第二步中，我们对内存变量求导:

\begin{align}
\dot{\zeta}_l(t) &= \omega_l \frac{\partial}{\partial t}\int_{-\infty}^t \epsilon(t') \cdot \exp(-\omega_l (t-t')) dt'\notag\\
&= \omega_l \biggl\{-\omega_l\int_{-\infty}^t \epsilon(t') \cdot \exp(-\omega_l (t-t')) dt' + \epsilon(t)\biggr\}\notag\\
&= \omega_l \biggl\{-\zeta_l(t) + \epsilon(t)\biggr\}\notag\\
\end{align}

通过这个技巧，我们可以摆脱时间卷积，但代价是额外的L个偏微分方程来估计内存变量，这些变量必须在有限差分代码中求解:

\begin{equation}
\dot{\zeta}_l(t) = \omega_l \epsilon(t) - \omega_l \zeta_l(t)\notag
\end{equation}

为了得到 **应力-速度** 公式中的偏微分方程，我们只需求一次导数，得到:

\begin{align}
\dot{\sigma}(t) &= \mu_u \cdot \dot{\epsilon}(t) - \sum_{l=1}^L \delta\mu a_l \xi_l(t),
\notag\\
\dot{\xi}_l(t) &= \omega_l \dot{\epsilon}(t) - \omega_l \xi_l(t)\notag
\end{align}

通过 $\xi_l = \dot{\zeta_l}$. 为了简化参数化，我们引入了无量纲弹性系数

\begin{equation}
Y_l = a_l \frac{\delta \mu}{\mu_u}\notag
\end{equation}

引出

\begin{equation}
\dot{\sigma}(t) = \mu_u \cdot \dot{\epsilon}(t) - \sum_{l=1}^L \mu_u Y_l \xi_l(t).
\notag
\end{equation}

## 粘弹性一维SH问题的运动方程

最后，我们将动量方程与应力-应变关系结合到粘弹性一维SH问题的运动方程中。在替换了上节课的简化符号之后:

\begin{align}
\sigma &\rightarrow \sigma_{yx}\notag\\
\epsilon &\rightarrow \epsilon_{yx}\notag\\
\mu_u &\rightarrow 2\mu_u\notag\\
\end{align}

我们得到动量守恒:

\begin{equation}
\rho \dot{v}_y = \frac{\partial \sigma_{yx}}{\partial x} + f_y\notag
\end{equation}

和粘弹性应力-应变关系:

\begin{align}
\dot{\sigma_{yx}}(t) &= 2 \mu_u \dot{\epsilon_{yx}}(t) - \sum_{l=1}^L \mu_u Y_l \xi_l(t)\notag\\
\dot{\xi}_l(t) &= \omega_l \dot{\epsilon_{yx}}(t) - \omega_l \xi_l(t)\notag\\
\end{align}

非松弛剪切模量:

\begin{equation}
\mu_u = \frac{\mu_0}{1-\sum_{l=1}^L Y_l}\notag
\end{equation}

和

\begin{equation}
\dot{\epsilon}_{yx} = \frac{1}{2} \frac{\partial v_y}{\partial x}\notag
\end{equation}

利用这些方程，定义了一维粘弹性SH问题，并可在最后的TEW2课中采用交错网格有限差分方法求解。

## 我们学习到:

- 用傅里叶反变换从GMB的频率域复剪切模量估计时域松弛函数 $\Psi(t)$
- 时域粘弹性应力-应变关系引入了一个时间卷积，在每个时间步长和空间地下点都需要，这将显著增加有限建模的计算成本
- 我们可以通过引入记忆变量来摆脱时间卷积，代价是为每个麦克斯韦体增加一个偏微分方程
- 最后，编制了一维波在黏弹性SH介质中的传播运动方程